In [ ]:
import numpy as np
import pandas as pd

import tabula


In [ ]:
#df = tabula.read_pdf('baseball_letter.pdf', pages=3)

infilename = 'dates.text'

infile = open(infilename,'r')

for line in infile:
    #print(line)
    if line[0:4]=='Mens':
        print(line)
        


In [ ]:
import re

In [ ]:
import pandas as pd
import re
from datetime import datetime, timedelta

# Load the data
#file_path = "/mnt/data/dates.text"
file_path = "dates.text"

# Read file content
with open(file_path, "r") as file:
    raw_data = file.readlines()

# Process each line and extract details
game_entries = [line.strip() for line in raw_data if line.strip()]

# Define regex pattern for extracting game details
game_pattern = re.compile(
    r"Mens Baseball (Home|Away) vs ([A-Za-z\s\(\)]+?)(?:Thursday|Tuesday|Wednesday|Friday|Saturday|Sunday|Monday), (\d{2}/\d{2} \d{1,2}:\d{2} [APM]+)(?:Thursday|Tuesday|Wednesday|Friday|Saturday|Sunday|Monday), (\d{2}/\d{2} \d{1,2}:\d{2} [APM]+)"
)

# Extract game details
games = []
for entry in game_entries:
    match = game_pattern.findall(entry)
    for m in match:
        location = m[0]
        opponent = m[1].strip()
        start_datetime = m[2]
        end_datetime = m[3]

        # Convert to datetime with year 2025
        start_dt = datetime.strptime(start_datetime, "%m/%d %I:%M %p").replace(year=2025)
        end_dt = datetime.strptime(end_datetime, "%m/%d %I:%M %p").replace(year=2025)

        # Store in structured format
        games.append({
            "Location": location,
            "Opponent": opponent,
            "Start DateTime": start_dt,
            "End DateTime": end_dt
        })

# Convert to DataFrame
games_df = pd.DataFrame(games)

# Filter for "Away" games
away_games_df = games_df[games_df["Location"] == "Away"]

# Define class schedule (Tuesdays and Thursdays from 9:40 AM to 11:10 AM)
class_times = [
    {"Day": "Tuesday", "Start": timedelta(hours=9, minutes=40), "End": timedelta(hours=11, minutes=10)},
    {"Day": "Thursday", "Start": timedelta(hours=9, minutes=40), "End": timedelta(hours=11, minutes=10)},
]

# Identify missed classes
missed_classes = []
for _, game in away_games_df.iterrows():
    game_start = game["Start DateTime"]
    game_end = game["End DateTime"]

    # Iterate over class times to check if the game overlaps
    for class_time in class_times:
        class_day = class_time["Day"]

        # Generate potential class dates within the range of the game
        current_date = game_start
        while current_date <= game_end:
            if current_date.strftime("%A") == class_day:
                class_start = timedelta(hours=class_time["Start"].seconds // 3600, minutes=class_time["Start"].seconds // 60 % 60)
                class_end = timedelta(hours=class_time["End"].seconds // 3600, minutes=class_time["End"].seconds // 60 % 60)
                game_time = timedelta(hours=current_date.hour, minutes=current_date.minute)

                if game_time <= class_end:
                    missed_classes.append({
                        "Date": current_date.date(),
                        "Day": class_day,
                        "Opponent": game["Opponent"],
                        "Game Start": game["Start DateTime"].strftime("%I:%M %p"),
                        "Game End": game["End DateTime"].strftime("%I:%M %p"),
                    })
            current_date += timedelta(days=1)

# Convert to DataFrame
missed_classes_df = pd.DataFrame(missed_classes)

## Display the missed classes
#import ace_tools as tools
#tools.display_dataframe_to_user(name="Corrected Missed Classes", dataframe=missed_classes_df)

missed_classes_df


In [ ]:
# Adjust the year to 2025 in the parsed dates
for game in games:
    game["Start DateTime"] = game["Start DateTime"].replace(year=2025)
    game["End DateTime"] = game["End DateTime"].replace(year=2025)

# Convert updated games list to DataFrame
games_df = pd.DataFrame(games)

# Filter for "Away" games
away_games_df = games_df[games_df["Location"] == "Away"]

# Identify missed classes again with the correct year
missed_classes = []

for _, game in away_games_df.iterrows():
    game_start = game["Start DateTime"]
    game_end = game["End DateTime"]

    # Iterate over class times to check if the game overlaps
    for class_time in class_times:
        class_day = class_time["Day"]

        # Generate potential class dates within the range of the game
        current_date = game_start
        while current_date <= game_end:
            if current_date.strftime("%A") == class_day:
                class_start = timedelta(hours=class_time["Start"].seconds // 3600, minutes=class_time["Start"].seconds // 60 % 60)
                class_end = timedelta(hours=class_time["End"].seconds // 3600, minutes=class_time["End"].seconds // 60 % 60)
                game_time = timedelta(hours=current_date.hour, minutes=current_date.minute)

                if game_time <= class_end:
                    missed_classes.append({
                        "Date": current_date.date(),
                        "Day": class_day,
                        "Opponent": game["Opponent"],
                        "Game Start": game["Start DateTime"].strftime("%I:%M %p"),
                        "Game End": game["End DateTime"].strftime("%I:%M %p"),
                    })
            current_date += timedelta(days=1)

# Convert to DataFrame
missed_classes_df = pd.DataFrame(missed_classes)

# Display the corrected missed classes
tools.display_dataframe_to_user(name="Corrected Missed Classes", dataframe=missed_classes_df)
